In [1]:
import pandas as pd
import re

>* We are going back to week 5 when we analyzed social media data to build edges between users. We will use the data from week 3, which is collected from Politician's Twitter and Facebook accounts. We will use the data to build a social network graph and analyze the network.

In [2]:
data=pd.read_csv('../week3/Political-media-DFE.csv', encoding='latin1')

In [3]:
data.columns

Index(['_unit_id', '_golden', '_unit_state', '_trusted_judgments',
       '_last_judgment_at', 'audience', 'audience:confidence', 'bias',
       'bias:confidence', 'message', 'message:confidence', 'orig__golden',
       'audience_gold', 'bias_gold', 'bioid', 'embed', 'id', 'label',
       'message_gold', 'source', 'text'],
      dtype='object')

>* Let's subset the DataFrame to have `label`, `source`, and `text` columns

In [4]:
df=data[['label', 'source', 'text']]

>* With this DataFrame, we want to extract the name of the user from the `label` column and create a new column called `username` in the DataFrame.

In [5]:
pattern=re.compile(r'(.+?)\s*\(')
df['username']=df['label'].str.replace('From: ', '').str.findall(pattern).str[0]

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_2046/1246208881.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username']=df['label'].str.replace('From: ', '').str.findall(pattern).str[0]


>* Let's lowercase the `username` column

In [7]:
df['username']=df['username'].str.lower()

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_2046/3231511176.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username']=df['username'].str.lower()


> * Let's extract the mentions from the `text` column and create a new column `mention` with the mentions.

In [9]:
pattern = re.compile(r'@[a-zA-Z0-9]+')
df['mention']=df['text'].apply(lambda x: pattern.findall(x))
df['mention']=df['mention'].apply(lambda x: [y.lower() for y in x])

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_2046/704253825.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mention']=df['text'].apply(lambda x: pattern.findall(x))
/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_2046/704253825.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['mention']=df['mention'].apply(lambda x: [y.lower() for y in x])


>* We also don't want to forget to convert the text into lowercase.

In [10]:
df['text']=df['text'].str.lower()

/var/folders/3_/r8z51q092v566bd_r0g_zj640000gn/T/ipykernel_2046/473300915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text']=df['text'].str.lower()


In [11]:
df.head(5)

,label,source,text,username,mention
0,From: Trey Radel (Representative from Florida),twitter,rt @nowthisnews: rep. trey radel (r- #fl) slam...,trey radel,[@nowthisnews]
1,From: Mitch McConnell (Senator from Kentucky),twitter,video - #obamacare: full of higher costs and ...,mitch mcconnell,[]
2,From: Kurt Schrader (Representative from Oregon),twitter,please join me today in remembering our fallen...,kurt schrader,[]
3,From: Michael Crapo (Senator from Idaho),twitter,rt @senatorleahy: 1st step toward senate debat...,michael crapo,[@senatorleahy]
4,From: Mark Udall (Senator from Colorado),twitter,.@amazon delivery #drones show need to update ...,mark udall,[@amazon]
